In [ ]:
import json
import numpy as np
def load_groundTruth():
    anns = json.load(open("../annotations.json"))
    categories = [c['name'] for c in anns['categories']]
    ids = []
    file_names = []
    imagesDic = {}
    for x in anns['images']:
        path = "../"+x['path']
        file_names.append(path)
        ids.append(x['id'])
        imagesDic[x['id']] = path
    file_names = np.asarray(file_names)
    #restrict ther the number of filenames to 10
    ids = np.asarray(ids)
    boards=[[] for _ in range(len(file_names))]
    for piece in anns['annotations']['corners']:
        idx = np.where(ids == piece['image_id'])[0][0]
        conors = piece['corners']
        boards[idx]= conors#["chessred2k"]
    split_ids = np.asarray(anns['splits']['test']['image_ids']).astype(int)

    intersect = np.isin(ids, split_ids)
    split_ids = np.where(intersect)[0]
    file_names = file_names[split_ids]
    file_names = file_names#[:45]
    #boards = boards[split_ids]
    ids = ids[split_ids]
    splitImages = []
    splitGroundTruth = []
    for id in ids:
        splitImages.append(imagesDic[id])
        splitGroundTruth.append(boards[id])
    #print(f"Number of {partition} images: {len(file_names)}")
    return splitImages ,splitGroundTruth
splitImages ,splitGroundTruth = load_groundTruth()



In [ ]:

# Flatten keypoints and normalize
import cv2
image = cv2.imread(splitImages[i])
img_height, img_width = image.shape[:2]
for i in range (len(splitImages)):
    conors = splitGroundTruth[i]
    keypoints = []
    for point in conors.values():
        x, y = point[0] / img_width, point[1] / img_height
        keypoints.extend([x, y, 2])  # 2 = visible and labeled

    # Class ID (0 for "square")
    class_id = 0

    # Save to .txt file
    with open("square_keypoints.txt", "w") as f:
        line = f"{class_id} " + " ".join([f"{kp:.6f}" for kp in keypoints])
        f.write(line)

In [ ]:
import cv2
for i in range (len(splitImages)):
    
    image = cv2.imread(splitImages[i])
    conors = splitGroundTruth[i]
    points = {k: [int(x), int(y)] for k, v in conors.items() for x, y in [v]}

    # Assign colors to each position (BGR format in OpenCV)
    colors = {
        'top_left': (0, 0, 255),      # Red
        'top_right': (0, 255, 0),     # Green
        'bottom_left': (255, 0, 0),   # Blue
        'bottom_right': (0, 255, 255) # Yellow
    }

    # Create a white image (adjust width/height as needed)
    width, height = 3000, 3000  # Large enough to fit your points

    # Draw the square (connect points with lines)
    cv2.line(image, tuple(points['top_left']), tuple(points['top_right']), (0, 0, 0), 2)
    cv2.line(image, tuple(points['top_left']), tuple(points['bottom_left']), (0, 0, 0), 2)
    cv2.line(image, tuple(points['top_right']), tuple(points['bottom_right']), (0, 0, 0), 2)
    cv2.line(image, tuple(points['bottom_left']), tuple(points['bottom_right']), (0, 0, 0), 2)

    # Draw the points with their respective colors
    for pos, (x, y) in points.items():
        cv2.circle(image, (x, y), 15, colors[pos], -1)  # -1 fills the circle
        cv2.putText(image, pos, (x + 20, y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
    image = cv2.resize(image,(720,720),image)
    # Display the image
    cv2.imshow("Square with Colored Points", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Save the image (optional)
    cv2.imwrite("square_with_colored_points.jpg", image)

In [ ]:

import torch
from ultralytics import YOLO

model = YOLO("yolo11n-pose.pt") 
# Train the model
#d:/A_University/ano4/semester_2/Vision/computer-vision/processed/res2k.yaml
model.train(    
    data="./res2k.yaml",
    epochs=100,  # Increased epochs
    imgsz=640,  # Reduced image size
    batch=8,  # Increased batch size
    workers=2,  # Enable multiprocessing
    name="Yolo11m_v2",
    fliplr=0.0,  # Disable flips
    flipud=0.0,
    degrees=0.0,  # Disable rotation
    perspective=0.0,  # Disable perspective
    optimizer="AdamW",  # More stable optimizer
    lr0=0.001,  # Learning rate
    warmup_epochs=3,  # Gradual learning start
    box=7.5,  # Adjust loss weights
    pose=12.0,
    kobj=1.0,
    overlap_mask=False,)